#Kaggle competition

Int this notebook i try make solution for competition https://www.kaggle.com/c/plant-pathology-2021-fgvc8

I created CNN, but  haven't possibility to make full solution, because haven't computer power for calculate.
Maybe in future i'll finish this project

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/plant-pathology-2021-fgvc8.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/ap_data')

OSError: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.models as models
import torchvision
from torchvision import transforms
import os
import torch
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
len(os.listdir('/content/ap_data/train_images'))

18348

In [ ]:
data = pd.read_csv('/content/ap_data/train.csv')

In [ ]:
exist_img = os.listdir('/content/ap_data/train_images')

In [ ]:
data.head()

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex


In [ ]:
data['exist'] = data.image.map(lambda x: 1 if x in exist_img else 0)

In [ ]:
data = data.drop(index=data[data.exist==0].index)

In [ ]:
data.head()

,image,labels,exist
0,800113bb65efe69e.jpg,healthy,1
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex,1
2,80070f7fb5e2ccaa.jpg,scab,1
3,80077517781fb94f.jpg,scab,1
4,800cbf0ff87721f8.jpg,complex,1


In [ ]:
new_lab = {key:val for val, key in  enumerate(data.labels.unique())}

In [ ]:
data.labels = data.labels.map(new_lab)

In [ ]:
data.head()

,image,labels,exist
0,800113bb65efe69e.jpg,0,1
1,8002cb321f8bfcdf.jpg,1,1
2,80070f7fb5e2ccaa.jpg,2,1
3,80077517781fb94f.jpg,2,1
4,800cbf0ff87721f8.jpg,3,1


In [ ]:
transform = transforms.Compose([
 transforms.Resize(512),
 transforms.ToTensor(),
 transforms.Normalize(mean=[0.485, 0.456, 0.406],
 std=[0.229, 0.224, 0.225])
 ])

In [ ]:
train_data = data.sample(frac=0.9)
valid_data = data.drop(index=train_data.index)

In [ ]:
train_data.to_csv('train_data.csv')
valid_data.to_csv('valid_data.csv')

In [ ]:
class APPLE(Dataset):
  def __init__(self, path_csv, img_path, transform=None):
    self.df = pd.read_csv(path_csv)
    self.img_path = img_path
    self.transform = transform

  def __getitem__(self, idx):
    image_name = os.path.join(self.img_path, self.df.iloc[idx, 1])
    image = Image.open(image_name)
    labels = self.df.iloc[idx, 2]
    if transform:
      img = self.transform(image)

    return img.to('cuda'), torch.tensor(labels).to('cuda')
  
  def __len__(self):
    return len(self.df)

In [ ]:
train_data = APPLE(path_csv='/content/train_data.csv', transform=transform, 
                   img_path='/content/ap_data/train_images')
valid_data = APPLE(path_csv='/content/valid_data.csv', transform=transform, 
                   img_path='/content/ap_data/train_images') 

In [ ]:
train_loader = DataLoader(train_data, shuffle=True)
valid_loader = DataLoader(valid_data, shuffle=True)

In [ ]:
import torchvision.models as models

In [ ]:
model = models.resnet152(pretrained=True)

In [ ]:
# for name, param in model.named_parameters():
#   param.requires_grad = False

In [ ]:
model.fc = nn.Sequential(nn.Linear(model.fc.in_features,500), nn.ReLU(),
nn.Dropout(), nn.Linear(500,12))

In [ ]:
model.to('cuda')

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import torch.optim as optim

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
! pip install pytorch-ignite

In [ ]:
device = 'cuda'

In [ ]:
from ignite.engine import Engine, _prepare_batch
from ignite.engine import Events

def process_function(engine, batch):
    model.train()
    optimizer.zero_grad()
    x, y = _prepare_batch(batch, device=device)
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    return loss.item()

trainer = Engine(process_function)

log_interval = 50

@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    iteration = (engine.state.iteration - 1) % len(train_loader) + 1
    if iteration % log_interval == 0:
        print("Epoch[{}] Iteration[{}/{}] Loss: {:.4f}"
              .format(engine.state.epoch, 
                         iteration, 
                         len(train_loader), 
                         engine.state.output))
        

In [ ]:
from ignite.metrics import Loss, Precision, Recall, Accuracy
from ignite.engine import create_supervised_evaluator
from torch.utils.data.dataset import Subset

In [ ]:
metrics = {
    'avg_loss': Loss(criterion),
    'avg_accuracy': Accuracy(),
    'avg_precision': Precision(average=True),
    'avg_recall': Recall(average=True)
}

In [ ]:
train_evaluator = create_supervised_evaluator(model, metrics=metrics, device='cuda')
val_evaluator = create_supervised_evaluator(model, metrics=metrics, device='cuda')

In [ ]:
indices = np.arange(len(train_data))
random_indices = np.random.permutation(indices)[:len(valid_data)]
train_subset = Subset(train_data, indices=random_indices)

train_eval_loader = DataLoader(train_subset, batch_size=100, shuffle=True,
                                num_workers=2, 
                                drop_last=True, pin_memory="cuda")

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_offline_train_metrics(engine):
    epoch = engine.state.epoch
    print("Compute train metrics...")
    metrics = train_evaluator.run(train_eval_loader).metrics
    print("Training Results - Epoch: {}  Average Loss: {:.4f} | Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, 
                      metrics['avg_loss'], 
                      metrics['avg_accuracy'], 
                      metrics['avg_precision'], 
                      metrics['avg_recall']))

@trainer.on(Events.EPOCH_COMPLETED)
def compute_and_display_val_metrics(engine):
    epoch = engine.state.epoch
    print("Compute validation metrics...")
    metrics = val_evaluator.run(val_loader).metrics
    print("Validation Results - Epoch: {}  Average Loss: {:.4f} | Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f}"
          .format(engine.state.epoch, 
                      metrics['avg_loss'], 
                      metrics['avg_accuracy'], 
                      metrics['avg_precision'], 
                      metrics['avg_recall']))

In [ ]:
output = trainer.run(train_loader, max_epochs=10)

Epoch[1] Iteration[50/16513] Loss: 2.3600
Epoch[1] Iteration[100/16513] Loss: 4.0979
Epoch[1] Iteration[150/16513] Loss: 2.1222
Epoch[1] Iteration[200/16513] Loss: 2.2093
Epoch[1] Iteration[250/16513] Loss: 0.9957
Epoch[1] Iteration[300/16513] Loss: 1.8636
Epoch[1] Iteration[350/16513] Loss: 0.7623
Epoch[1] Iteration[400/16513] Loss: 2.3460
Epoch[1] Iteration[450/16513] Loss: 1.5107
Epoch[1] Iteration[500/16513] Loss: 1.5540
Epoch[1] Iteration[550/16513] Loss: 2.0263
Epoch[1] Iteration[600/16513] Loss: 2.2737
Epoch[1] Iteration[650/16513] Loss: 1.7729
Epoch[1] Iteration[700/16513] Loss: 2.3673
Epoch[1] Iteration[750/16513] Loss: 2.4415
Epoch[1] Iteration[800/16513] Loss: 2.0723
Epoch[1] Iteration[850/16513] Loss: 3.3700
Epoch[1] Iteration[900/16513] Loss: 3.2265
Epoch[1] Iteration[950/16513] Loss: 2.3616
Epoch[1] Iteration[1000/16513] Loss: 3.4423
Epoch[1] Iteration[1050/16513] Loss: 1.8415
Epoch[1] Iteration[1100/16513] Loss: 2.3246
Epoch[1] Iteration[1150/16513] Loss: 2.3358
Epoch[1]

Current run is terminating due to exception: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataset.py", line 330, in __getitem__
    return self.dataset[self.indices[idx]]
  File "<ipython-input-54-cc987a77d32d>", line 14, in __getitem__
    return img.to('cuda'), torch.tensor(labels).to('cuda')
  File "/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py", line 161, in _lazy_init
    "Cannot re-in

RuntimeError: ignored